In [109]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from pathlib import Path

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')

try:
    driver = webdriver.Chrome(options=chrome_options)
    # driver = webdriver.Chrome()
except:
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    # driver = webdriver.Chrome(service=service)

df = pd.read_excel('buscas.xlsx')
df.loc[0, "Termos banidos"] = 'mini watch 11 xr'
df.loc[0, "Nome"] = 'iphone 12 64 gb'

display(df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch 11 xr,3000,3500
1,rtx 3060,zota galax,4000,4500


In [110]:
def google_search(driver, product_name, forbidden_words, min_price, max_price):

    driver.get('https://www.google.com/')
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(product_name, Keys.ENTER)

    elements_list = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for element in elements_list:
            if 'shopping' in element.text.lower():
                element.click()
                break


    result_list = driver.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')

    # Dealing with input variables
    product_name = product_name.lower()
    product_name_list = product_name.split(" ")
    forbidden_words = forbidden_words.lower()
    forbidden_words_list = forbidden_words.split(" ")
    max_price = float(max_price)
    min_price = float(min_price)

    found_products = [] # the final result
    for result in result_list:
        name = result.find_element(By.CLASS_NAME, 'tAxDx').text
        name = name.lower()

        # Verifying the product name
        has_forbidden_words = False
        for word in forbidden_words_list:
            if word in name:
                has_forbidden_words = True
        
        # Ensure that the product name is the desired one
        has_all_product_names = True
        for word in product_name_list:
            if word not in name:
                has_all_product_names = False
                break
        
        if not has_forbidden_words and has_all_product_names:
            try:
                # Formating the price -> (from str to float)
                price = float(result.find_element(By.CLASS_NAME, 'a8Pemb').text.replace("R$ ", "").replace(".", "").replace(",", "."))
        
                # Checking if price is within the defined limit
                if min_price <= price <= max_price:
                    link = result.find_element(By.CSS_SELECTOR, 'a.shntl').get_attribute('href')
                    found_products.append((name, price, link))
            except:
                continue


    return found_products


In [114]:
found_products_df = pd.DataFrame()
for row in df.index:
    product_name = df.loc[row, "Nome"]
    forbidden_words = df.loc[row, "Termos banidos"]
    min_price = df.loc[row, "Preço mínimo"]
    max_price = df.loc[row, "Preço máximo"]

    final_results = google_search(driver, product_name, forbidden_words, min_price, max_price)
    if final_results:
        products_df = pd.DataFrame(final_results, columns=['Product', 'Price', 'Link'])
        found_products_df = pd.concat([found_products_df, products_df])

display(found_products_df)


,Product,Price,Link
0,iphone 12 preto 64gb,3333.00,https://www.google.com/url?url=https://rj.olx....
1,iphone 12 64gb - azul - estou zerado,3329.10,https://www.google.com/url?url=https://www.tro...
2,"(seminovos) iphone 12 apple preto, 64gb",3399.15,https://www.google.com/url?url=https://www.taq...
3,"(seminovos) iphone 12 apple branco, 64gb",3399.15,https://www.google.com/url?url=https://www.taq...
4,iphone 12 64gb branco seminovo novinho,3495.00,https://www.google.com/url?url=https://www.ztc...
5,"(seminovos) iphone 12 apple azul, 64gb",3399.15,https://www.google.com/url?url=https://www.taq...
0,pc gamer t-gamer thorn amd ryzen 5 4500 / nvid...,4141.44,https://www.google.com/url?url=https://www.ter...
1,yeston rtx3060-12g d6 placa gráfica 192 bits/g...,4325.08,https://www.google.com/url?url=https://www.ame...


In [ ]:
# product_name = 'iphone 12 64 gb'
# forbidden_words = 'mini watch 11 xr'
# min_price = 3000
# max_price = 3500

# final_results = google_search(driver, product_name, forbidden_words, min_price, max_price)
# if final_results:
#     products_df = pd.DataFrame(final_results, columns=['Product', 'Price', 'Link'])
# else:
#     products_df = None

# display(products_df)
# for i in final_results:
#     print(i)

,Product,Price,Link
0,iphone 12 preto 64gb,3333.00,https://www.google.com/url?url=https://rj.olx....
1,iphone 12 64gb - azul - estou zerado,3329.10,https://www.google.com/url?url=https://www.tro...
2,"(seminovos) iphone 12 apple preto, 64gb",3399.15,https://www.google.com/url?url=https://www.taq...
3,"(seminovos) iphone 12 apple branco, 64gb",3399.15,https://www.google.com/url?url=https://www.taq...
4,iphone 12 64gb branco seminovo novinho,3495.00,https://www.google.com/url?url=https://www.ztc...
5,"(seminovos) iphone 12 apple azul, 64gb",3399.15,https://www.google.com/url?url=https://www.taq...


('iphone 12 preto 64gb', 3333.0, 'https://www.google.com/url?url=https://rj.olx.com.br/rio-de-janeiro-e-regiao/celulares/iphone-12-preto-64gb-1119236185&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiK-u-pr738AhXGr5UCHZysAFsQgOUECNwM&usg=AOvVaw1SSz6I0654bqmupoeS-KOv')
('iphone 12 64gb - azul - estou zerado', 3329.1, 'https://www.google.com/url?url=https://www.trocafy.com.br/iphone-12-64gb-azul-sou-como-novo-2552/p%3Fidsku%3D966%26srsltid%3DAeTuncrdiOzrvVchtDo3CLlDpQcV73nkDYLRMJBUl2y6hYVh_zF0PJ-XZt8&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiK-u-pr738AhXGr5UCHZysAFsQguUECO0M&usg=AOvVaw2qqFs08_mHb6vtKfBnTBki')
('(seminovos) iphone 12 apple preto, 64gb', 3399.15, 'https://www.google.com/url?url=https://www.taqi.com.br/seminovos-usado-apple-iphone-12-preto-nacional/221635%3Fsrsltid%3DAeTuncrwMiFKIlt991HV9V7BEqUUJ5uWUPNCRg4hiPj3odUjcyo_Z9nJGNc&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiK-u-pr738AhXGr5UCHZysAFsQgOUECKUP&usg=AOvVaw0MCeGEp-ni8eUjmwQuT7Hb')
('(seminovos) iphone 12 apple branco, 64gb', 3399.15, 'https://www.goog

In [ ]:
# product_name = 'rtx 3060'
# forbidden_words = 'zota galax'
# min_price = 4000
# max_price = 5500

# final_results = google_search(driver, product_name, forbidden_words, min_price, max_price)
# if final_results:
#     products_df = pd.DataFrame(final_results, columns=['Product', 'Price', 'Link'])
# else:
#     products_df = None

# display(products_df)
# for i in final_results:
#     print(i)